In [1]:
import numpy as np

In [2]:
class DataSet():
    def __init__(self):
        self.wrong_char_list = []
        self.correct_char_list = []
        self.wrong_word_list = []
        self.correct_word_list = []

In [3]:
class Text():
    def __init__(self):
        self.split = 20000
        self.train = DataSet()
        self.test = DataSet()
        self.distinc_char_list = []
        
    def init_param(self):
        # Number of unique characters
        self.charnum = len(self.distinc_char_list)
        # Initial state probabilities
        self.I = np.array((self.charnum))
        # Transition Probability Matrix
        self.A = np.array((self.charnum, self.charnum))
        # Emission Probability Matrix
        self.B = np.array((self.charnum, self.charnum))
        
    def run(self):
        # Wrong character list
        wr_list = self.test.wrong_char_list
        # Correct character list
        cr_list = self.test.correct_char_list
        # Probability for iteration
        pr = np.zeros((26))
        # Probability for previous iteration
        prp = pr
        char_count = len(cr_list)
        for i in range(char_count):
            c = ord(cr_list[i]) - 97
            w = ord(wr_list[i]) - 97
        
    def learn(self):
        # Wrong character list
        wr_list = self.train.wrong_char_list
        # Correct character list
        cr_list = self.train.correct_char_list
        # Number of the correct characters
        char_count = len(cr_list)
        # Starting character counts
        I_count = np.zeros((26))
        # Number of transitions
        A_count = np.zeros((26,26))
        # Number of emissions
        B_count = np.zeros((26,26))
        # Previous correct character
        pc = ""
        # Previous wrong character
        pw = ""
        for i in range(char_count):
            c = ord(cr_list[i]) - 97
            w = ord(wr_list[i]) - 97
            if c != -2:
                #print (i, pc)
                if i == 0 or pc == -2:
                    # Initial state probabilities
                    
                    I_count[c] += 1
                    #print ("Initial state c:", c, " ; I_c :", I_count[c] )
                else:
                    # Transition probabilities
                    A_count[pc,c] += 1
                    #print ("Transition c:", c, " ; pc: ", pc, " ; A_pc_c :", A_count[pc,c] )
                # Exit probabilities
                #print ("Exit c:", c, " ; w: ", w, " ; B_c_w :", B_count[c,w] )                
                B_count[c,w] += 1
            pc = c
            pw = w     
            
        self.I = I_count / I_count.sum()
        self.A = A_count / A_count.sum(axis=1, keepdims=True)
        self.B = B_count / B_count.sum(axis=1, keepdims=True)
                   
        
    def readData(self):
        wrong_word = ""
        correct_word = ""
        f = open('docs.data')
        i = 0
        data = self.test
        for line in f:
            if i == self.split:
                data = self.train
                wrong_word = ""
                correct_word = ""               
            s = line.strip().split(" ")
            if len(s) != 2:
                continue
            correct, wrong = s
            if wrong == "_":
                if len(wrong_word) > 0:
                    data.wrong_word_list.append(wrong_word)
                wrong_word = ""
            else:
                wrong_word += wrong
                if wrong not in self.distinc_char_list:
                    self.distinc_char_list.append(wrong)
            if correct == "_":
                if len(correct_word) > 0:
                    data.correct_word_list.append(correct_word)        
                correct_word = ""
            else:
                correct_word += correct
                if correct not in self.distinc_char_list:
                    self.distinc_char_list.append(correct)                
            data.wrong_char_list.append(wrong)
            data.correct_char_list.append(correct)
            i += 1
            
    def printI(self):
        for i in range(len(self.I)):
            print (chr(i + 97) , self.I[i])
            
    def printA(self):
        s = "A"
        for i in range(26):
            s += ";" + chr(i+97)
        print (s)
        for i in range(26):
            s = chr(i + 97)
            for j in range(26):
                s += "; %.4f" % self.A[i,j]
            print (s)        
        
        
    def printB(self):
        s = "B"
        for i in range(26):
            s += ";" + chr(i+97)
        print (s)
        for i in range(26):
            s = chr(i + 97)
            for j in range(26):
                s += "; %.4f" % self.B[i,j]
            print (s)          

In [4]:
class Viterbi():
    def __init__(self, visible_states, hidden_states, I, A, B):
        self.visible_states = visible_states
        self.hidden_states = hidden_states
        self.I = I
        self.A = A
        self.B = B
        self.hscnt = len(hidden_states)
        self.vscnt = len(visible_states)
    
    def run(self, observations):
        obcnt = len(observations)
        T = np.zeros((obcnt, self.hscnt))
        result = []
        for i in range(obcnt):
            c = observations[i]
            vidx = self.visible_states.index(c)
            b = B[:,vidx]

            if i == 0:
                Umax = self.I
            else:
                U = T[i-1,:] * A.T
                Umax = U.max(axis=1)
            G = Umax * b
            T[i,:] = G
            pidx = G.argmax()
            predict = self.hidden_states[pidx]
            result.append(predict)
            pc = c
            i = i + 1

        self.T = T
        return result
            
        

In [5]:
t = Text()
t.readData()
t.learn()

In [6]:
hs = [chr(i + 97) for i in range(26)]
vs = [chr(i + 97) for i in range(26)]
I = t.I
A = t.A
B = t.B
w = Viterbi(vs, hs, I, A, B)

wc = np.zeros((2,2))
cc = np.zeros((2,2))
for word, cword in zip(t.test.wrong_word_list, t.test.correct_word_list):
    o = list(word)
    prediction = w.run(o)
    prediction = "".join(prediction)
    print (word, ";", prediction, ";", cword )
    if word == cword:
        wc[0,0] += 1
        cc[0,0] += len(word)
    else:
        wc[0,1] += 1
        for j in range(len(word)):
            if word[j] == cword[j]:
                cc[0,0] += 1
            else:
                cc[0,1] += 1
    if prediction == cword:
        wc[1,0] += 1
        cc[1,0] += len(prediction)
    else:
        wc[1,1] += 1
        for j in range(len(prediction)):
            if prediction[j] == cword[j]:
                cc[1,0] += 1
            else:
                cc[1,1] += 1



by ; by ; by
their ; their ; their
own ; own ; own
acvount ; account ; account
violence ; violence ; violence
is ; is ; is
for ; for ; for
them ; them ; them
q ; a ; a
form ; form ; form
of ; of ; of
pjberation ; puberation ; liberation
in ; in ; in
othef ; othef ; other
words ; words ; words
by ; by ; by
committing ; committing ; committing
violebce ; violence ; violence
they ; they ; they
bteqj ; btequ ; break
thrpugh ; thepugh ; through
the ; the ; the
psychologjcal ; psychological ; psychological
resttaints ; resttaints ; restraints
that ; that ; that
have ; have ; have
been ; been ; been
trained ; trained ; trained
into ; into ; into
them ; them ; them
befause ; befause ; because
they ; they ; they
are ; are ; are
oversocialized ; oversocialized ; oversocialized
thesr ; these ; these
restdaints ; resteaints ; restraints
have ; have ; have
been ; been ; been
more ; more ; more
confining ; confining ; confining
for ; for ; for
them ; them ; them
than ; than ; than
for ; for ; for
ot

In [7]:
print (wc)
print (wc / wc.sum(axis=1, keepdims=True))
print (cc)
print (cc / cc.sum(axis=1, keepdims=True))

[[ 2121.  1290.]
 [ 2363.  1048.]]
[[ 0.62181179  0.37818821]
 [ 0.69275872  0.30724128]]
[[ 14934.   1654.]
 [ 15258.   1330.]]
[[ 0.90028937  0.09971063]
 [ 0.91982156  0.08017844]]


In [8]:
t.printI()

a 0.105908789509
b 0.0500994621675
c 0.0418846238857
d 0.0285493258675
e 0.0299123259412
f 0.0350327856774
g 0.0162086495248
h 0.0358063803139
i 0.0808590584248
j 0.0017682163118
k 0.00287335150667
l 0.0241656229279
m 0.0411847049289
n 0.0251234067634
o 0.0810800854638
p 0.0517571649598
q 0.00103145951521
r 0.0265600825168
s 0.074559787814
t 0.173690414794
u 0.0101672437928
v 0.00478891917778
w 0.054741029986
x 0.0
y 0.00224710822957
z 0.0


In [9]:
t.printA()

A;a;b;c;d;e;f;g;h;i;j;k;l;m;n;o;p;q;r;s;t;u;v;w;x;y;z
a; 0.0000; 0.0246; 0.0433; 0.0304; 0.0001; 0.0054; 0.0229; 0.0005; 0.0296; 0.0026; 0.0214; 0.1438; 0.0198; 0.2101; 0.0003; 0.0195; 0.0002; 0.0937; 0.0721; 0.1770; 0.0170; 0.0347; 0.0029; 0.0004; 0.0268; 0.0007
b; 0.0357; 0.0021; 0.0005; 0.0000; 0.4202; 0.0005; 0.0000; 0.0000; 0.0346; 0.0098; 0.0000; 0.1519; 0.0005; 0.0000; 0.0615; 0.0000; 0.0000; 0.0367; 0.0083; 0.0062; 0.1297; 0.0047; 0.0000; 0.0000; 0.0972; 0.0000
c; 0.1656; 0.0000; 0.0164; 0.0000; 0.1407; 0.0000; 0.0000; 0.1681; 0.1442; 0.0000; 0.0169; 0.0242; 0.0000; 0.0000; 0.1503; 0.0000; 0.0018; 0.0301; 0.0034; 0.1017; 0.0310; 0.0000; 0.0000; 0.0000; 0.0055; 0.0000
d; 0.0411; 0.0000; 0.0009; 0.0100; 0.3323; 0.0005; 0.0041; 0.0005; 0.1826; 0.0037; 0.0000; 0.0100; 0.0023; 0.0055; 0.1465; 0.0005; 0.0000; 0.0347; 0.0634; 0.0000; 0.1123; 0.0283; 0.0027; 0.0000; 0.0183; 0.0000
e; 0.0536; 0.0022; 0.0668; 0.0819; 0.0387; 0.0320; 0.0093; 0.0062; 0.0206; 0.0002; 0.0010; 0.0471; 0.0542;

In [10]:
t.printB()

B;a;b;c;d;e;f;g;h;i;j;k;l;m;n;o;p;q;r;s;t;u;v;w;x;y;z
a; 0.8993; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0340; 0.0000; 0.0337; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0330
b; 0.0000; 0.8911; 0.0000; 0.0000; 0.0000; 0.0000; 0.0205; 0.0349; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0221; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0313; 0.0000; 0.0000; 0.0000; 0.0000
c; 0.0000; 0.0000; 0.9020; 0.0237; 0.0000; 0.0261; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0233; 0.0000; 0.0248; 0.0000; 0.0000
d; 0.0000; 0.0000; 0.0161; 0.8962; 0.0221; 0.0242; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0225; 0.0000; 0.0000; 0.0000; 0.0000; 0.0189; 0.0000; 0.0000
e; 0.0000; 0.0000; 0.0000; 0.0335; 0.8997; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000; 0.0000;